In [21]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np

In [22]:
date_cols = [
    'Auftragseingang', 
    'Auftragsende_SOLL',
    'AFO_Start_SOLL',
    'AFO_Ende_SOLL',
    'AFO_Start_IST',
    'AFO_Ende_IST'
]

data = pd.read_csv(
    '../data/processed/data_cleaned_timestamp_1.csv',
    parse_dates=date_cols,
    sep=',',
    low_memory=False
)

In [23]:
data.head()

,AuftragsID,BauteilID,Bauteilbezeichnung,Auftragseingang,Priorität,Auftragsende_SOLL,Arbeitsschritt,Arbeitsschrittbezeichnung,AFO_Start_SOLL,AFO_Ende_SOLL,AFO_Start_IST,AFO_Ende_IST,MaschinenID,Maschinenbezeichnung,Maschinenkapazität
0,95a859f51cf541e0b4aed3a38bb93065,1,Steuerventilmodul,2013-09-20,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN
1,ed4e40cb93c04d0f9bcb8f7ecdc8752a,1,Steuerventilmodul,2013-11-09,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN
2,ce233ad078b9429b8bd40f09100e8ee0,1,Steuerventilmodul,2013-12-30,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN
3,c6b0430e1b7b4f328f0ac195c3070390,1,Steuerventilmodul,2013-10-05,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN
4,5a5b4b41d6d246cfbe862018b557702b,1,Steuerventilmodul,2013-07-27,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528238 entries, 0 to 1528237
Data columns (total 15 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   AuftragsID                 1528238 non-null  object        
 1   BauteilID                  1528238 non-null  int64         
 2   Bauteilbezeichnung         1528238 non-null  object        
 3   Auftragseingang            1528238 non-null  datetime64[ns]
 4   Priorität                  1528238 non-null  int64         
 5   Auftragsende_SOLL          1528238 non-null  datetime64[ns]
 6   Arbeitsschritt             1528238 non-null  int64         
 7   Arbeitsschrittbezeichnung  1528238 non-null  object        
 8   AFO_Start_SOLL             1528238 non-null  datetime64[ns]
 9   AFO_Ende_SOLL              1528238 non-null  datetime64[ns]
 10  AFO_Start_IST              1528238 non-null  datetime64[ns]
 11  AFO_Ende_IST               1528238 no

In [25]:
from pandas.tseries.offsets import BDay, CustomBusinessHour

# Arbeitszeit: Montag–Freitag, 07:00–15:00 Uhr
cbh = CustomBusinessHour(start='07:00', end='15:00', weekmask='Mon Tue Wed Thu Fri')

In [26]:
def arbeitszeit_diff_fast(start, end):
    if pd.isna(start) or pd.isna(end):
        return np.nan
    if end < start:
        return 0

    # Ganze Tage dazwischen (Mo–Fr)
    bdays = np.busday_count(start.date(), end.date(), weekmask='1111100')
    
    # Stundenanteil des Starttags
    start_hour = start.hour + start.minute/60
    end_hour = end.hour + end.minute/60

    start_hours = max(0, 15 - max(7, start_hour)) if start_hour < 15 else 0
    end_hours = max(0, min(15, end_hour) - 7) if end_hour > 7 else 0

    total_hours = bdays * 8 + end_hours - (8 - start_hours)
    return max(total_hours, 0)

In [ ]:
data["AFO_Dauer_SOLL_Stunden"] = data.apply(
    lambda x: arbeitszeit_diff_fast(x["AFO_Start_SOLL"], x["AFO_Ende_SOLL"]), axis=1
)

data["AFO_Dauer_IST_Stunden"] = data.apply(
    lambda x: arbeitszeit_diff_fast(x["AFO_Start_IST"], x["AFO_Ende_IST"]), axis=1
)

data["AFO_Verspätung_Arbeitszeit"] = (
    data["AFO_Dauer_IST_Stunden"] - data["AFO_Dauer_SOLL_Stunden"]
)

data["Start_Delay_Arbeitszeit"] = data.apply(
    lambda x: arbeitszeit_diff_fast(x["AFO_Start_SOLL"], x["AFO_Start_IST"]), axis=1
)

data["End_Delay_Arbeitszeit"] = data.apply(
    lambda x: arbeitszeit_diff_fast(x["AFO_Ende_SOLL"], x["AFO_Ende_IST"]), axis=1
)
 

In [28]:
# 1️⃣ Auftragsebenen-Daten erzeugen: erster & letzter Arbeitsschritt + Start/Ende
auftrag_zeiten = (
    data.groupby("AuftragsID")
        .agg({
            "Arbeitsschritt": ["min", "max"],         # erster & letzter Schritt
            "AFO_Start_IST": "min",                  # erster tatsächlicher Start
            "AFO_Ende_IST": "max"                    # letzter tatsächlicher Abschluss
        })
        .reset_index()
)

# Spalten umbenennen für Klarheit
auftrag_zeiten.columns = [
    "AuftragsID",
    "AFO_Erster_Schritt",
    "AFO_Letzter_Schritt",
    "AFO_Start_IST_Erster",
    "AFO_Ende_IST_Letzter"
]

# 2️⃣ Gesamtlaufzeit in Kalendertagen (IST)
auftrag_zeiten["Auftrags_Laufzeit_IST_Tage"] = (
    (auftrag_zeiten["AFO_Ende_IST_Letzter"] - auftrag_zeiten["AFO_Start_IST_Erster"]).dt.total_seconds() / (24 * 3600)
)

# 3️⃣ Merge zurück in Hauptdatensatz
data = data.merge(
    auftrag_zeiten[[
        "AuftragsID",
        "AFO_Erster_Schritt",
        "AFO_Letzter_Schritt",
        "AFO_Start_IST_Erster",
        "AFO_Ende_IST_Letzter",
        "Auftrags_Laufzeit_IST_Tage"
    ]],
    on="AuftragsID",
    how="left"
)


In [32]:
data.describe()

,BauteilID,Auftragseingang,Priorität,Auftragsende_SOLL,Arbeitsschritt,AFO_Start_SOLL,AFO_Ende_SOLL,AFO_Start_IST,AFO_Ende_IST,Maschinenkapazität,AFO_Dauer_SOLL_Stunden,AFO_Dauer_IST_Stunden,AFO_Verspätung_Arbeitszeit,Start_Delay_Arbeitszeit,End_Delay_Arbeitszeit,AFO_Erster_Schritt,AFO_Letzter_Schritt,AFO_Start_IST_Erster,AFO_Ende_IST_Letzter,Auftrags_Laufzeit_IST_Tage
count,1.528238e+06,1528238,1.528238e+06,1528238,1.528238e+06,1528238,1528238,1528238,1528238,538654.000000,1.528238e+06,1.528238e+06,1.528238e+06,1.528238e+06,1.528238e+06,1.528238e+06,1.528238e+06,1528238,1528238,1.528238e+06
mean,1.630314e+00,2018-11-10 17:44:39.287912448,1.405400e+00,2019-02-11 18:58:59.749673216,1.509230e+02,2019-02-10 17:21:38.321779712,2019-02-10 21:10:02.713203200,2019-04-03 23:26:54.162532608,2019-04-04 02:01:31.282497024,11.040187,1.252179e+00,8.175974e-01,-4.345812e-01,2.998268e+02,2.995212e+02,1.271833e+00,9.623879e+02,2019-02-11 02:44:02.435235328,2019-07-29 23:12:59.072893184,1.688534e+02
min,1.000000e+00,2000-01-01 00:00:00,1.000000e+00,2013-05-30 09:02:00,1.000000e+00,2013-05-30 08:31:00,2013-05-30 09:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,1.000000,1.666667e-02,1.666667e-02,-5.300000e+01,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+02,2014-01-01 07:00:00,2014-01-01 07:04:00,6.944444e-04
25%,1.000000e+00,2016-03-08 00:00:00,1.000000e+00,2016-06-14 10:12:00,2.200000e+01,2016-06-13 12:11:00,2016-06-13 13:12:00,2016-08-02 07:01:00,2016-08-02 07:31:00,8.000000,5.000000e-01,5.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,9.990000e+02,2016-06-13 07:00:00,2016-11-22 11:55:00,1.222917e+00
50%,2.000000e+00,2018-10-11 00:00:00,1.000000e+00,2019-01-04 11:02:00,1.000000e+02,2019-01-02 07:31:00,2019-01-02 13:42:00,2019-03-14 10:00:00,2019-03-14 10:43:00,8.000000,5.000000e-01,5.000000e-01,0.000000e+00,2.116667e+00,2.016667e+00,1.000000e+00,9.990000e+02,2019-01-01 07:00:00,2019-08-08 14:10:00,2.240361e+02
75%,2.000000e+00,2021-06-10 00:00:00,2.000000e+00,2021-09-07 10:12:00,1.000000e+02,2021-09-06 14:21:00,2021-09-07 07:01:00,2021-11-12 09:37:30,2021-11-12 10:16:45,16.000000,8.333333e-01,1.083333e+00,8.333333e-02,2.040000e+01,1.918333e+01,1.000000e+00,9.990000e+02,2021-09-06 07:00:00,2022-04-26 11:06:00,2.962042e+02
max,3.000000e+00,2024-08-01 00:00:00,3.000000e+00,2024-08-05 07:22:00,9.990000e+02,2024-08-02 14:31:00,2024-08-02 14:32:00,2024-08-02 14:49:00,2024-08-02 15:00:00,25.000000,5.400000e+01,8.666667e+00,7.833333e+00,2.387950e+03,2.387667e+03,9.990000e+02,9.990000e+02,2024-08-02 07:00:00,2024-08-02 15:00:00,4.210882e+02
std,5.469001e-01,NaN,6.667985e-01,NaN,2.801966e+02,NaN,NaN,NaN,NaN,5.239962,3.412710e+00,8.790481e-01,3.381687e+00,6.095608e+02,6.090777e+02,3.642632e+00,1.776903e+02,NaN,NaN,1.384225e+02


In [33]:
data.to_csv('../data/processed/data_feature_3.csv', index=False)